# Import Packages

In [89]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
# load .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [91]:
import os
import json
import sys
from loguru import logger
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [92]:
# Import writer module
from source import writer
from source import utils
from source import evaluator

In [94]:
career_story_path = os.path.join(os.getcwd(), '..', 'data', 'career_story.txt')
career_story = utils.load_career_story(career_story_path)

jd_path = os.path.join(os.getcwd(), '..', 'data', 'clio_jd.txt')
jd = utils.load_job_description(jd_path)

# Generate CV V1

In [95]:
customized_criteria = evaluator.gen_customized_cv_criteria(jd=jd, model='o1')

In [96]:

cv_v1 = writer.generate_cv(career_story=career_story, jd=jd, customized_criteria=customized_criteria, model='o1')

In [97]:
cv_v1_path = os.path.join(os.getcwd(), '..', 'output', 'cv_v1.html')
utils.string_to_html_file(html_string=cv_v1, output_filepath=cv_v1_path)

2025-03-22 15:52:10.768 | SUCCESS  | source.utils:string_to_html_file:12 - HTML file created successfully at: /Users/phamanhdung/Documents/genius-cv/notebook/../output/cv_v1.html


# Improve to CV V2

by looping n times to improve missing points compared to the customized criteria

In [98]:
def fullfill_missing_customized_criteria(cv: str) -> str:
    missing_criteria = evaluator.check_missing_criteria(customized_cv_criteria=customized_criteria, cv_string=cv, model='gpt-4o-mini')
    missing_criteria = json.loads(missing_criteria)['missing_criteria']
    if missing_criteria is None:
        logger.info('All criteria are met')
        return cv
    else:
        cv_v2 = cv
        for i in range(3):
            logger.info(f'Improve round {i+1}')
            cv_v2 = writer.rewrite_cv(cv=cv_v2, career_story=career_story, improvement_points=missing_criteria, model='gpt-4o')
            missing_criteria = evaluator.check_missing_criteria(customized_cv_criteria=customized_criteria, cv_string=cv_v2, model='gpt-4o-mini')
            if missing_criteria is None:
                logger.info('All criteria are met')
                break
        return cv_v2, missing_criteria

In [99]:
cv_v2, missing_criteria = fullfill_missing_customized_criteria(cv=cv_v1)
logger.warning(f'Still missing criteria: {missing_criteria}')

2025-03-22 15:52:14.266 | INFO     | __main__:fullfill_missing_customized_criteria:10 - Improve round 1
2025-03-22 15:52:31.411 | INFO     | __main__:fullfill_missing_customized_criteria:10 - Improve round 2
2025-03-22 15:52:52.044 | INFO     | __main__:fullfill_missing_customized_criteria:10 - Improve round 3
2025-03-22 15:53:11.320 | WARNING  | __main__:<module>:2 - Still missing criteria: {
  "missing_criteria": [
    "Criterion about clarifying the scope of experience in data science is not met because while the CV states '5+ years applied experience in Data Science', it does not detail specific projects, industries, or business impacts associated with this experience.",
    "Criterion about demonstrable experience in analytics for product & user behavior is not met because although the CV mentions retention analysis and churn prediction, it does not clearly spell out how these analyses were performed or the direct outcomes they influenced.",
    "Criterion about proven ability to 

In [100]:
cv_v2_path = os.path.join(os.getcwd(), '..', 'output', 'cv_v2.html')
utils.string_to_html_file(html_string=cv_v2, output_filepath=cv_v2_path)

2025-03-22 15:53:15.750 | SUCCESS  | source.utils:string_to_html_file:12 - HTML file created successfully at: /Users/phamanhdung/Documents/genius-cv/notebook/../output/cv_v2.html


# Improve CV V3 

Spelling by country

In [87]:
spelling_improvement = evaluator.check_country_spelling(cv=cv_v2, country="Canada", model='gpt-4o')
logger.warning(f'Spelling improvement: {spelling_improvement}')
cv_v3 = writer.rewrite_cv(cv=cv_v2, career_story=career_story, improvement_points=spelling_improvement, model='gpt-4o')

2025-03-22 15:10:05.360 | WARNING  | __main__:<module>:2 - Spelling improvement: ```json
{
  "spelling_errors": [
    "Word 'program' should be 'programme' in Canadian English",
    "Word 'behavior' should be 'behaviour' in Canadian English",
    "Word 'modeling' should be 'modelling' in Canadian English",
    "Word 'catalog' should be 'catalogue' in Canadian English",
    "Word 'analyze' should be 'analyse' in Canadian English",
    "Word 'utilize' should be 'utilise' in Canadian English",
    "Word 'focused' should be 'focussed' in Canadian English"
  ]
}
```


In [88]:
cv_v3_path = os.path.join(os.getcwd(), '..', 'output', 'cv_v3.html')
utils.string_to_html_file(html_string=cv_v3, output_filepath=cv_v3_path)

2025-03-22 15:10:26.206 | SUCCESS  | source.utils:string_to_html_file:12 - HTML file created successfully at: /Users/phamanhdung/Documents/genius-cv/notebook/../output/cv_v3.html
